`homework_nsmc_251230.ipynb`에 작성하세요. # 95

위 파일을 다운로드 받고, 지침에 따라 감성분석 모델 구현 및 학습/평가/추론 코드를 작성하세요.

# 🧪 실습 가이드: 네이버 영화리뷰 감성분석

## 🎯 실습 목표
- 한글 텍스트 데이터 전처리 과정을 이해한다.
- 토크나이징과 시퀀스 변환의 필요성을 설명할 수 있다.
- PyTorch 기반 감성분석 모델을 구현하고 학습한다.
- 학습된 모델로 문장 감성 추론을 수행한다.
- Embedding Layer의 의미를 시각적으로 이해한다.



## 01. 데이터 로드

**✅ 실습 과제**
- 네이버 영화 리뷰 데이터셋을 불러온다.
    https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
- 데이터의 컬럼 구성과 샘플을 확인한다.

**🔍 확인 질문**
1. 리뷰 텍스트와 정답 라벨은 각각 어떤 컬럼에 저장되어 있는가?
2. 긍정 / 부정 라벨은 어떤 값으로 표현되어 있는가?



# 2. 데이터 전처리

### 02-01. 한글 전처리

**✅ 실습 과제**
- 특수문자, 숫자, 불필요한 기호를 제거한다.
- 정규표현식을 사용하여 한글만 남긴다.

**🔍 확인 질문**
1. 한글 전처리를 하지 않으면 어떤 문제가 발생할 수 있는가?
2. 감성 분석에서 이모지나 느낌표는 제거하는 것이 항상 옳은가?




### 02-02. Tokenizing & Sequencing

**✅ 실습 과제**
- Tokenizer를 사용해 단어 사전을 생성한다.
- 문장을 정수 시퀀스로 변환한다.
- padding을 적용하여 시퀀스 길이를 맞춘다.

### 🔍 확인 질문
1. `num_words` 파라미터는 어떤 역할을 하는가?
2. padding을 하지 않으면 배치 학습에서 어떤 문제가 발생하는가?



### 02-03. Sequence Decoding

**✅ 실습 과제**
- 정수 시퀀스를 다시 텍스트로 복원해본다.
- 토큰 인덱스와 단어의 매핑 관계를 확인한다.

**🔍 확인 질문**
1. `<OOV>` 토큰은 언제 사용되는가?
2. 디코딩 결과가 원문과 완전히 동일하지 않은 이유는 무엇인가?


## 03. 모델 생성 및 학습

**✅ 실습 과제**
- Embedding Layer를 포함한 감성분석 모델을 정의한다.
- 손실 함수와 옵티마이저를 설정한다.
- 학습 과정을 통해 loss 변화를 확인한다.

**🔍 확인 질문**
1. Embedding Layer는 어떤 역할을 하는가?
2. One-hot 인코딩 대신 Embedding을 사용하는 이유는 무엇인가?



## 04. 모델 평가

**✅ 실습 과제**
- 검증 데이터로 모델 성능을 평가한다.
- 정확도(acc)와 손실(loss)을 확인한다.

**🔍 확인 질문**
1. 학습 데이터 성능과 평가 데이터 성능 차이가 의미하는 것은 무엇인가?
2. 과적합 여부는 어떻게 판단할 수 있는가?



## 05. 모델 추론

**✅ 실습 과제**
- 임의의 문장을 입력하여 감성을 예측한다.
- 출력 확률을 기반으로 긍/부정을 해석한다.

**🔍 확인 질문**
1. 모델 출력값은 확률인가 점수인가?
2. 기준값(threshold)은 왜 필요한가?
